In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
from pandas.plotting import table
import sys
from tabulate import tabulate
import os
import string
from time import sleep
import matplotlib.pyplot as plt
import dataframe_image as dfi

listaforn = [] 
forn = ''
        
def biel(stri, xi='X'):
    '''Retorna o index do ultimo X no nome '''
    xi = xi.upper()
    try:
        qtd = stri[::-1].upper().index(xi)
    except ValueError:
        return False
    qt = len(stri) - qtd
    return qt

    
def maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire=''):
    '''Função maluco pega todos os valores do produto do xml e retorna o preço final
    escolhendo a qtd de caixa com base na escolha do usuário e no nome do produto
    val1 vira valo1 qdo só tem números, ele pega o lado esquerdo, do ultimo X encontrado
    val2 vira valo2 qdo só tem números, ele pega o lado direito,  do ultimo X encontrado '''
    valo1, valo2 = '', ''
    val1 = nome[x - 5:x]
    val2 = nome[x:x + 5]
    for i in val1:
        if i.isdigit():
            valo1 += i
    for i in val2:
        if i.isdigit():
            valo2 += i
    if dire == 'e':
        if not valo1:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo1) * qtrib)
        return pf, valo1
    if dire == 'd':
        if not valo2:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
        return pf, valo2
    if dire == 'nda':
        if not valo2:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
        return pf, valo2
    if dire[0] == '0':
        if dire[1:4].lower() == 'esq':
            pf = (vprod + icms + fcp + ipi - desc) / (int(valo1) * qtrib)
            return pf, valo1
        if dire[1:4].lower() == 'dir':
            pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
            return pf, valo2


def formato(nome, pf, ean, qcx=1,cfop = '51'):
    global itens
    pv = '{:.2f}'.format(float(pf))
    for i in itens:
        if i['nome'] == nome:
            i['preco'] = pv
            i['ean'] = ean
            i['qcx'] = qcx
            if cfop == '99':
                i['cfop'] = cfop
            else:
                i['cfop'] = '51'


def digitos(cx):
    new, qcx = '', 0
    for i in cx:
        if i.isdigit():
            new += i
    try:
        qcx = int(new)
    except ValueError:
        pass
    return qcx


def escolha(ucom, utrib, ean):
    ''' Função para escolher qual melhor método de
    calcular os preços finais e achar as qtd na caixa'''
    ut = utrib[0:2].lower()
    uti = utrib[0:3].lower()
    qcx = checkEan(ean)
    if ucom and utrib and ucom.lower() == utrib.lower() and utrib.isalpha() == False:
        return 'cx24utrib'
    elif ucom and utrib and ucom.lower() == utrib.lower() and ucom.isalpha() == False:
        return 'cx24ucom'
    elif ut == 'un' or uti == 'und' or uti == 'uni' or ut == 'pc' or ut == 'fr' or ut == 'kg' or ut == 'qt' or ut == 'gr'\
            or uti == 'lat' or ut == 'lt' or ut == 'pa' or ut == 'cp' or ut == 'pt' or uti == 'pec':
        return 'un'
    elif qcx and int(qcx) >= 2:
        qcx = int(digitos(qcx))
        return "ean", qcx
    elif (ucom.lower() != ut) and (ut.strip() == 'un' or ut.strip() == 'pc'):
        return 'utrib*ucom'
    else:
        return 'maluco'


def names(rooti):
    '''Função para pegar as partes do xml
    e fazer o tratamento de dados necessário'''
    listannf = []
    global qcx, px, pv, itens, resp, maximo, listaforn, diaok
    nome, pf, x, utrib, bo, new, nnf, xfant, = '', 0, 0, '', False, '', '', ''
    listaforn, itens, fcp, ean, qcx, ucom, boc, dire, inf, jaSabe = [], [], 0, 0, 0, '', 0, '', '', False
    pv, ipi, icms, vprod, desc, qtrib, dia, cfop = 0, 0, 0, 0, 0, 0,'', ''
    for child in rooti.iter():
        bo = False
        try:
            if child.tag[36:] == 'dhSaiEnt' or child.tag[36:] == 'dhEmi':
                dia = child.text
            if child.tag[36:] == 'nNF':
                nnf = child.text
            if child.tag[36:] == 'xFant':
                xfant = child.text
                listaforn.append(xfant[:20])
                if child.text[:9].lower() == 'nova camp':
                    jaSabe = True
                    resp = 'maluco'
                    dire = 'esq'
                if child.text == 'MARSIL':
                    jaSabe = True
                    resp = 'marsil'
            if child.tag[36:] == 'xProd':
                ucom, utrib = '', ''
                desc, icms, vprod, pf, ipi, fcp, qcx = 0, 0, 0, 0, 0, 0, 0
                nome = child.text
                itens.append({'nome': nome})
            if child.tag[36:] == 'CFOP':
                cfop = child.text
                lista_boni = ['5910','5911','5912','5912','5913','5914',
                          '5915','5916','5917','5918','5919','1201',
                          '1202','1203','1204','1208','1209','1212',
                          '1410','1411','1503','1504','1505','1506',
                          '1553','1660','1661','1662','1918','1919',
                          '2201','2202','2203','2204','2208','2209',
                          '2212','2410','2411','2503','2504','2505',
                          '2506','2553','2660','2661','2662','2918',
                          '2919','5201','5202','5208','5209','5210',
                          '5410','5411','5412','5413','5503','5553',
                          '5555','5556','5660','5661','5662','5918',
                          '5919','5921','6201','6202','6208','6209',
                          '6210','6410','6411','6412','6413','6503',
                          '6553','6555','6556','6660','6661','6662',
                          '6918','6919','6921','7201','7202','7210',
                          '7211','7212','7553','7556']
                if cfop in lista_boni:
                    cfop = '99'
                else:
                    cfop = '51'
            if child.tag[36:] == 'vProd':
                vprod = float(child.text)
            if child.tag[36:] == 'cEANTrib':
                ean = (child.text)
            if child.tag[36:] == 'uCom':
                ucom = child.text
            if child.tag[36:] == 'uTrib':
                utrib = child.text
            if child.tag[36:] == 'qTrib':
                qtrib = float(child.text)
            if child.tag[36:] == 'vDesc':
                desc = float(child.text)
            if child.tag[36:] == 'vICMSST':
                icms = float(child.text)
            if child.tag[36:] == 'vFCPST':
                fcp = float(child.text)
            if child.tag[36:] == 'vIPI':
                ipi = float(child.text)
            if child.tag[36:] == 'infAdProd':
                inf = child.text
            if child.tag[36:39] == 'det' or child.tag[36:] == 'total':
                diaok = dia[8:10]+'/'+dia[5:7]+'/'+dia[:4]
                bo = True if boc else False
                boc = True
            if bo:
                if not jaSabe:
                    resp = escolha(ucom, utrib, ean)
                    print('Escolha foi ', resp)
                    if checkEan(ean) and int(checkEan(ean)) > 1:
                        qcx = checkEan(ean)
                print('qcx = ', qcx)
                if resp == 'marsil':
                    qcx = int(nome[-6:-2])
                    pf = (vprod + icms + fcp - + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx, cfop)
                    qcx = 0
                if resp[0] == 'ean':
                    pf = (vprod + icms + fcp + ipi - desc) / (resp[1] * qtrib)
                    formato(nome, pf, ean, qcx, cfop)
                    qcx = 0
                if qcx and ucom != utrib and utrib.lower().strip() == "un":
                    pf = (vprod + icms + fcp + ipi - desc) / (int(qcx) * qtrib)
                    formato(nome, pf, ean, qcx, cfop)
                    qcx = 0
                if resp == 'else':
                    pf = (vprod + icms + fcp + ipi - desc) / qtrib
                    formato(nome, pf, ean, cfop=cfop)
                if resp == 'maluco':
                    if not jaSabe:
                        dire = 'nda'
                    if dire == 'nda' or not dire:
                        y = biel(inf)
                        if not y:
                            resp = 'un'
                        else:
                            qcx = maluco(icms, desc, qtrib, vprod,
                                         fcp, ipi, dire='nda', x=int(y), nome=inf)
                            if not qcx == 'un':
                                pf = qcx[0]
                                qcx = qcx[1]
                                formato(nome, pf, ean, qcx, cfop)
                            else:
                                pf = (vprod + icms - desc + fcp + ipi) / qtrib
                                formato(nome, pf, ean, cfop=cfop)
                    elif dire == 'dir':
                        x = biel(nome)
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire="d")
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx, cfop)
                    elif dire == 'esq':
                        x = biel(nome)
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire="e")
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx, cfop)
                    elif dire[0] == '0':
                        x = biel(nome, xi=dire[-1])
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire)
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx, cfop)
                    else:
                        break
                if resp == 'cx24utrib':
                    qcx = digitos(utrib)
                    pf = (vprod + icms + fcp + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx, cfop)
                    qcx = 0
                if resp == 'cx24ucom':
                    qcx = digitos(ucom)
                    pf = (vprod + icms + fcp + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx, cfop)
                    qcx = 0
                if resp == 'un':
                    pf = (vprod + icms - desc + fcp + ipi) / qtrib
                    formato(nome, pf, ean ,cfop=cfop)
                if resp == 'utrib*ucom':
                    pf = (vprod + icms - desc + fcp + ipi) / (qtrib)  # <<----- era qcom
                    formato(nome, pf, ean, cfop=cfop)
            if child.tag[36:] == 'total':
                return itens
        except ZeroDivisionError or IndexError or KeyError:
            continue


def organiza(lista, ordenador, *retorno):
    '''Função para organizar os preços
    por ordem de preço crescente'''
    for i in range(len(lista)):
        lista[i]['nome'] = lista[i]['nome'].replace(",", ";").replace(":", ";").replace("  ", " ")
        lista[i]['nome'] = lista[i]['nome'].replace("  ", " ").replace("  ", " ").replace("  ", " ")
    aux = []
    for dct in lista:
        l = []
        for i in retorno:
            l.append(str(dct[i]))
        plv = "{} : {}".format(dct[ordenador], ','.join(l))
        aux.append(plv)
    aux.sort(key=lambda aux: list(map(float, re.findall(r'\d+', aux))))
    resultado = []
    for i in aux:
        resultado.append(i.split(':')[1].strip().split(','))
    return resultado


def preco(itens):
    '''formata o texto:
    nome do item ========= R$ 25.00'''
    di, lisnome, lispre = {}, [], []
    deu = organiza(itens, 'preco', 'nome', 'preco', 'ean', 'qcx', 'cfop')
    for i in deu:
        try:
            n = i[0]
            l = 68 - len(str(n))
            p = (i[1])
            lisnome.append(n)
            if i[4] == '99':
                lispre.append(f'boni')
            else:
                lispre.append(f'R${p}')
        except:
            lisnome.append('nome')
            lispre.append('preco')
    with open("log.txt", 'a') as log:
        for k in deu:
            if int(k[3]) >= 2:
                log.write(f"\n{k}")
    di['DESCRICAO DO ITEM'] = lisnome
    di['PRECO'] = lispre
    return di


def init(ean=0,mes='04',loja=['0','1','2','3','4','5','6'],idz='usuarioteste',numero=0):
    '''função de inicialização'''
    print('entrei init')
    global listaforn
    try:
        os.mkdir(f'C:/Users/pdv/projetos_jupyter/retorno/{idz}')
    except:
        pass
    purgedir(f'C:/Users/pdv/projetos_jupyter/retorno/{idz}')
    xmls,lista,lojitas,nnfs, arq = [],[],[],[],[]
    cnpjs = ['05349537000185','07246178000166','08158815000105','32265622000138',
             '05349537000266','23017342000209','23017342000110']
    lojitas = [cnpjs[int(i)] for i in loja]
    for cnpj in lojitas:
        dire = f'N:/XML ENTRADA/destinadas/{cnpj}/2022-{mes}'
        if ean:
            print(dire)
            arq = [f for f in os.listdir(dire) if str(ean) in f]
        elif numero:
            arq = [f for f in os.listdir(dire) if str(numero) in f]
        for i in arq:
            xmls.append({f'{cnpj}':i})
    for k in xmls: 
        for c,v in k.items():
            print(v[25:34])
#             if (numero and str(numero) in v[25:34]) or ean:
            lista.append(f'N:/XML ENTRADA/destinadas/{c}/2022-{mes}/{v}')
            x = (v[25:34])
            nnfs.append('NF: ' + x)
    ki=0
    k = 0
    for i in lista:
        lis = []
        try:
            lis.append(nnfs[k])
            tree = ET.parse(i)
            rooti = tree.getroot()
            listaforn = []
            di = preco(names(rooti))
            lis.append(f'{diaok} . . . . . . . {lojases(i[26:40])}')
            df = pd.DataFrame(di, columns=['DESCRICAO DO ITEM', 'PRECO'])
            abc = [['CHAVE NFE', i[49:93]]]
            pf = pd.DataFrame(abc, columns = lis)
            pf = pf.rename_axis(None)
            dfi.export(pf, f'retorno/{idz}/nota{ki}.png')
            ki += 1
            dfi.export(df, f'retorno/{idz}/nota{ki}.png')
            ki += 1
            k += 1
        except Exception as e: print(e)

            
def purgedir(parent):
    for root, dirs, files in os.walk(parent):                                      
        for item in files:
            filespec = os.path.join(root, item)
            os.unlink(filespec)
            
            
def checkEan(ean):
    ean = str(ean)
    if len(ean) > 13:
        ean = ean[1:]
    with open("txte.txt", 'r') as db:
        for linha in db:
            if linha[:13] == ean[:13]:
                return linha[15:]
            
            
def lojases(i):
    cnpjs = ['05349537000185','07246178000166','08158815000105','32265622000138',
             '05349537000266','23017342000209','23017342000110']
    loj = ['LJ1 - GUANABARA', 'LJ2 - PRIMAVERA', 'LJ3 - IGUATEMI', 'LJ4 - FLAMBOYANT', 'LJ5 - SÃO PAULO', 'LJ6 - VALINHOS', 'LJ7 - OURO VERDE']
    return loj[cnpjs.index(i)]

        
if __name__ == '__main__':
    x = init(ean = '00153705', mes='04', idz='2094003673', loja= ['2'])
    print(x)

entrei init
pastas ok
antes do for ['08158815000105']
N:/XML ENTRADA/destinadas/08158815000105/2022-04
verdade?  ['35220400153705000300550030009975621214730652.xml', '35220400153705000300550030009985271176449969.xml', '35220400153705000300550030009985301819146462.xml', '35220400153705000300550030010002021934788851.xml', '35220400153705000300550030010004841934788858.xml', '35220400153705000300550030010009551392063440.xml', '35220400153705000300550030010018701565005263.xml', '35220400153705000300550030010021921793415095.xml', '35220400153705000300550030010027271185519611.xml', '35220400153705000300550030010037141337008360.xml', '35220400153705000300550030010037181911360503.xml']
000997562
000998527
000998530
001000202
001000484
001000955
001001870
001002192
001002727
001003714
001003718
Escolha foi  maluco
qcx =  0
Escolha foi  ('ean', 10)
qcx =  10 

Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qcx =  0
Escolha foi  maluco
qc

In [1]:
import os
ean = '06204131'
dire = 'N:/XML ENTRADA/destinadas/08158815000105/2022-04'
arq = [f for f in os.listdir(dire) if str(ean) in f]
print(arq)

['35220406204131001904550010008559891479222245.xml', '35220406204131001904550010008614101217181442.xml']
